In [19]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)



cpu


In [20]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [21]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [22]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n: ]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[58, 71,  2,  3,  1, 57, 58, 56],
        [ 1, 28, 68, 71, 68, 73, 61, 78],
        [58, 67,  0, 58, 77, 54, 66, 62],
        [58,  1, 61, 62, 66,  1, 67, 62]])
targets: 
tensor([[71,  2,  3,  1, 57, 58, 56, 65],
        [28, 68, 71, 68, 73, 61, 78, 11],
        [67,  0, 58, 77, 54, 66, 62, 67],
        [ 1, 61, 62, 66,  1, 67, 62, 67]])


In [23]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [25]:
import torch
import torch.nn as nn  # Corrected import statement

class BigramLanguageModel(nn.Module):  # Corrected 'Module' capitalization
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # Fixed '==' to '='

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None: 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index if (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars  = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)
    
        


tr)U00NN_&0eFqJJrF[sQ;sOS7j"QF[;NG-0o4.MTtChu30"﻿E- 1[Mh;zCl9CyU7o1]Gq2yuKVE )KQh;:'NG9P!EG.eoT*orv:J4!EG6L'k"**33Dd &rvLF]GhQNXO
poE,yjv_;*:fz9t!Jm﻿VlQcI2J;tckfgDQs?XsEEc:)Gq;.DdKMNs3CKtFjvVIWVr-ua'tE:OV1Dk[[﻿Fq1H8c_B_qm_
5nFU6-zCQ2v:Ctt,_NfVlglG:;JzHl.w*"*ZjF﻿Y
M0inpRmUKzk[jYx1]7Jx:]R14yHEV(:B﻿;)c,TVlkTVlEPn
Y-_;:;g]Vd(of"k[;?e&ULv:I-ywOTBloV1
'
lg_4z7i-lWv3t yU6L3D481"]lg*c
IU6J5C4aD[jxlWeqYYNGie,"[Y.e-]x'& (,pljAwXNd[ULbQ[XNUI_V]]l0HzCysfW*lh0;58(K:(Mv!IGtR)D7-IExWVRSC﻿;﻿ij!IwF0(k[MNZiQLQi50


In [26]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data 
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# After the training loop is complete, print the final loss
print("Final Loss:", loss.item())


step: 0, train loss: 4.8027, val loss: 4.8295
step: 250, train loss: 4.7481, val loss: 4.7384
step: 500, train loss: 4.6837, val loss: 4.6859
step: 750, train loss: 4.6258, val loss: 4.6222
Final Loss: 4.485805988311768


In [27]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


14ZuFobukaQ[RkL:Yesm"w* SF
pmQFHc;&I-WNqZ_.*"I_GyfVUXhbQeVt 1_W.IT:OSatR4QnMvA'39]2w.mIM!eB,UKvAMiGCWlWecR2?F﻿5)W0p?rSmaQj6z3 4QVUCmYZtR5lMNvF:4.uh8 ;J'(1[B[f&AfqqStKGfoYnAPXiLpZ2?TV,jIZ)UOcY-WtqmM,nhXNA'iPR1m3F
IK6S2Aae-IrzC﻿;ttRZjet?z7-tRStr4YRQvGa_PRG﻿_6w_m(]]'mT" mEuW]0Zn-Rt,*[MNGQnf9HJzCr:0UR﻿_!egqVl0xfUoV:o3F[xpkEK]auNMKvN5TQqO8(i&Bm-yFvkTF
9R[3:uKr
K)z2d3gFOMka*o:L289uK.;Zo:Ef'iEc0nNAKQRGXNd(vAKv6:v*oK.uC)TrMcT(ioAbFL)V;(kqgw*:Im7UMn_*DY;CW﻿Sj?tTE.mHUI-YajVloHB[LCmmuCnkrCEv)ef[CyneB1plKrO


In [28]:
 input = torch.rand((4, 8, 10))
B, T, C = input.shape
output = input.view(B*T, C)
print(output)
print(output[:, -1, :])

tensor([[4.8510e-01, 5.1724e-01, 8.5226e-01, 2.8527e-01, 2.4550e-01, 4.9355e-01,
         1.2512e-01, 8.6134e-01, 8.1473e-01, 9.7961e-01],
        [2.8156e-01, 1.1220e-01, 8.7697e-01, 5.1171e-01, 7.1902e-02, 9.9586e-01,
         1.4384e-01, 9.7513e-01, 6.8499e-02, 5.1727e-01],
        [8.8060e-01, 5.6563e-01, 5.1454e-01, 2.5161e-02, 1.3449e-01, 1.2796e-01,
         9.5049e-01, 9.4436e-01, 7.7836e-01, 4.3470e-01],
        [1.2987e-01, 7.1475e-01, 1.0673e-01, 5.6218e-01, 6.1190e-01, 9.8412e-01,
         1.8980e-01, 7.3448e-01, 2.5681e-01, 3.5614e-01],
        [6.4268e-01, 9.6257e-01, 4.5303e-01, 9.6053e-01, 5.6976e-01, 7.4541e-01,
         9.6330e-01, 2.4614e-01, 2.0450e-01, 3.0819e-01],
        [9.2029e-02, 9.3134e-01, 2.4367e-01, 8.9908e-01, 9.9421e-01, 9.6307e-01,
         7.6896e-01, 2.8176e-01, 8.9815e-01, 5.0224e-01],
        [1.0308e-01, 5.9224e-01, 2.3607e-01, 9.6122e-01, 7.3988e-01, 5.8066e-01,
         8.7374e-01, 6.0019e-01, 9.0173e-01, 4.5659e-01],
        [2.7006e-01, 7.7029

IndexError: too many indices for tensor of dimension 2

In [31]:
x = torch.tensor([1], dtype=torch.float32)
y = F.tanh(x)
print(y)

tensor([0.7616])
